In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from esm.pretrained import esmfold_v1 as meta_esmfold_v1
meta_esmfold = meta_esmfold_v1()

from plaid.esmfold import esmfold_v1
from plaid.esmfold import output_to_pdb

plaid_esmfold = esmfold_v1()

In [8]:
import torch
device = torch.device("cuda:0") 
plaid_esmfold = plaid_esmfold.to(device)
meta_esmfold = meta_esmfold.to(device)

In [9]:
plaid_esmfold = plaid_esmfold.eval().requires_grad_(False)
meta_esmfold = meta_esmfold.eval().requires_grad_(False)

In [10]:
seq = "AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQTLGQHDFSAGEGLYTHMKALRPDEDRLSPLHSVYVDQWDWERVMGDGERQFSTLKSTVEAIWAGIKATEAAVSEEFGLAPFLPDQIHFVHSQELLSRYPDLDAKGRERAIAKDLGAVFLVGIGGKLSDGHRHDVRAPDYDDWSTPSELGHAGLNGDILVWNPVLEDAFELSSMGIRVDADTLKHQLALTGDEDRLELEWHQALLRGEMPQTIGGGIGQSRLTMLLLQLPHIGQVQAGVWPAAVRESVPSLL"
print(len(seq))

327


In [11]:
plaid_pred = plaid_esmfold.infer_pdb(seq)

In [12]:
meta_pred = meta_esmfold.infer_pdb(seq)

In [15]:
import py3Dmol
view = py3Dmol.view(width=400, height=400)
view.addModel(plaid_pred)
view.addModel(meta_pred)
view.setStyle({"model": 0}, {"cartoon": {"color": "green", "opacity": 0.5}})
view.setStyle({"model": 1}, {"cartoon": {"color": "red","opacity": 0.5}})
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Only using `sm_s`

In [9]:
output = plaid_esmfold.infer(seq)
print(output.keys())

dict_keys(['frames', 'sidechain_frames', 'unnormalized_angles', 'angles', 'positions', 'states', 's_initial', 's_after_layernorm', 'z_after_layernorm', 'single', 'sm_s', 'sm_z', 's_s', 's_z', 'distogram_logits', 'lm_logits', 'aatype', 'atom14_atom_exists', 'residx_atom14_to_atom37', 'residx_atom37_to_atom14', 'atom37_atom_exists', 'residue_index', 'lddt_head', 'plddt', 'ptm_logits', 'ptm', 'aligned_confidence_probs', 'predicted_aligned_error', 'max_predicted_aligned_error', 'mean_plddt', 'chain_index'])


In [10]:
print(output['s_initial'].shape)
print(output['sm_s'].shape)

torch.Size([8, 1, 327, 384])
torch.Size([1, 327, 384])


In [21]:
aa = output['aatype'].clone()
sm_s = output['sm_s'].clone()


In [33]:
structure = plaid_esmfold.from_sm_s(sm_s, true_aa=aa, no_recycles=10)
sm_s_pdb = output_to_pdb(structure)

In [40]:
from plaid import utils
utils.view_py3Dmol(sm_s_pdb[0])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# post layer-norm

In [11]:
output.keys()

dict_keys(['frames', 'sidechain_frames', 'unnormalized_angles', 'angles', 'positions', 'states', 's_initial', 's_after_layernorm', 'z_after_layernorm', 'single', 'sm_s', 'sm_z', 's_s', 's_z', 'distogram_logits', 'lm_logits', 'aatype', 'atom14_atom_exists', 'residx_atom14_to_atom37', 'residx_atom37_to_atom14', 'atom37_atom_exists', 'residue_index', 'lddt_head', 'plddt', 'ptm_logits', 'ptm', 'aligned_confidence_probs', 'predicted_aligned_error', 'max_predicted_aligned_error', 'mean_plddt', 'chain_index'])

In [12]:
print(output['s_after_layernorm'].shape)
# print(output['sm_s'].shape)

torch.Size([8, 1, 327, 384])


In [15]:
for k, v in output.items():
    try:
        print(k, v.shape)
    except:
        print(k)


frames torch.Size([8, 1, 327, 7])
sidechain_frames torch.Size([8, 1, 327, 8, 4, 4])
unnormalized_angles torch.Size([8, 1, 327, 7, 2])
angles torch.Size([8, 1, 327, 7, 2])
positions torch.Size([8, 1, 327, 14, 3])
states torch.Size([8, 1, 327, 384])
s_initial torch.Size([8, 1, 327, 384])
s_after_layernorm torch.Size([8, 1, 327, 384])
z_after_layernorm torch.Size([8, 1, 327, 327, 128])
single torch.Size([1, 327, 384])
sm_s torch.Size([1, 327, 384])
sm_z torch.Size([1, 327, 327, 128])
s_s torch.Size([1, 327, 1024])
s_z torch.Size([1, 327, 327, 128])
distogram_logits torch.Size([1, 327, 327, 64])
lm_logits torch.Size([1, 327, 23])
aatype torch.Size([1, 327])
atom14_atom_exists torch.Size([1, 327, 14])
residx_atom14_to_atom37 torch.Size([1, 327, 14])
residx_atom37_to_atom14 torch.Size([1, 327, 37])
atom37_atom_exists torch.Size([1, 327, 37])
residue_index torch.Size([1, 327])
lddt_head torch.Size([8, 1, 327, 37, 50])
plddt torch.Size([1, 327, 37])
ptm_logits torch.Size([1, 327, 327, 64])
ptm